<a href="https://colab.research.google.com/github/ViRiver24/Lesson-7/blob/main/lesson_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torchvision import datasets, transforms

# Завантаження та попередня обробка даних
transform = transforms.Compose([
    transforms.ToTensor(),                      # Перетворення в тензор
    transforms.Normalize((0.5,), (0.5,))       # Нормалізація до діапазону [-1, 1]
])

# Завантаження набору даних MNIST
mnist_dataset = datasets.MNIST(
    root='./data',          # Де зберігати дані
    train=True,             # Завантажити тренувальний набір
    transform=transform,    # Застосувати трансформацію
    download=True           # Завантажити, якщо немає локально
)

# Завантажувач даних (DataLoader)
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_dataset,
    batch_size=64,          # Розмір батчу
    shuffle=True            # Перемішування даних
)

# Перевірка
images, labels = next(iter(data_loader))
print(f"Форма даних: {images.shape}")
print(f"Мінімальне значення пікселя: {images.min()}")
print(f"Максимальне значення пікселя: {images.max()}")

Форма даних: torch.Size([64, 1, 28, 28])
Мінімальне значення пікселя: -1.0
Максимальне значення пікселя: 1.0


In [3]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super(Generator, self).__init__()

        # Архітектура генератора
        self.fc1 = nn.Linear(z_dim, 256)  # Перший Dense шар
        self.fc2 = nn.Linear(256, 512)    # Другий Dense шар
        self.fc3 = nn.Linear(512, 1024)   # Третій Dense шар
        self.fc4 = nn.Linear(1024, 28*28) # Вихідний Dense шар, розмір зображення 28x28

        # Функції активації
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, z):
        # Пропуск через шари з активаціями
        x = self.relu(self.fc1(z))   # Шар 1
        x = self.relu(self.fc2(x))   # Шар 2
        x = self.relu(self.fc3(x))   # Шар 3
        x = self.fc4(x)              # Вихідний шар

        # Перетворення в діапазон [-1, 1] за допомогою Tanh
        x = self.tanh(x)

        # Перетворення в зображення 28x28
        return x.view(-1, 1, 28, 28)

# Ініціалізація генератора
z_dim = 100  # Розмір вхідного вектора шуму
generator = Generator(z_dim)

# Перевірка розміру вихідного зображення
z = torch.randn(64, z_dim)  # Приклад випадкового шуму
generated_images = generator(z)

print(f"Форма зображень: {generated_images.shape}")

Форма зображень: torch.Size([64, 1, 28, 28])


In [4]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # Архітектура дискримінатора
        self.fc1 = nn.Linear(28*28, 1024)   # Перший Dense шар
        self.fc2 = nn.Linear(1024, 512)     # Другий Dense шар
        self.fc3 = nn.Linear(512, 256)      # Третій Dense шар
        self.fc4 = nn.Linear(256, 1)        # Вихідний шар (1 вихід для класифікації)

        # Функції активації
        self.leaky_relu = nn.LeakyReLU(0.2) # LeakyReLU з параметром негативного нахилу 0.2
        self.sigmoid = nn.Sigmoid()         # Sigmoid для виведення ймовірності

    def forward(self, x):
        # Перетворення зображення в одномірний вектор
        x = x.view(-1, 28*28)  # Перетворення з 28x28 в 784 елементи
        x = self.leaky_relu(self.fc1(x))  # Шар 1
        x = self.leaky_relu(self.fc2(x))  # Шар 2
        x = self.leaky_relu(self.fc3(x))  # Шар 3
        x = self.fc4(x)                   # Вихідний шар

        # Використовуємо sigmoid для отримання ймовірності
        x = self.sigmoid(x)

        return x

# Ініціалізація дискримінатора
discriminator = Discriminator()

# Перевірка розміру вихідного результату
sample_image = torch.randn(64, 1, 28, 28)  # Приклад випадкових зображень (batch_size=64)
output = discriminator(sample_image)

print(f"Форма виходу: {output.shape}")

Форма виходу: torch.Size([64, 1])


In [5]:
import torch
import torch.optim as optim
import torch.nn as nn

# Функція втрат: binary cross-entropy
criterion = nn.BCELoss()

# Ініціалізація оптимізаторів для генератора та дискримінатора
lr = 0.0002  # Коефіцієнт навчання

# Оптимізатор для генератора
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

# Оптимізатор для дискримінатора
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Перевірка ініціалізації оптимізаторів та функції втрат
print("Оптимізатор для генератора:", optimizer_g)
print("Оптимізатор для дискримінатора:", optimizer_d)
print("Функція втрат:", criterion)

Оптимізатор для генератора: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0002
    maximize: False
    weight_decay: 0
)
Оптимізатор для дискримінатора: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0002
    maximize: False
    weight_decay: 0
)
Функція втрат: BCELoss()


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Перевірка наявності GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Завантаження MNIST даних
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
data_loader = DataLoader(mnist_dataset, batch_size=64, shuffle=True)

# Ініціалізація моделей генератора та дискримінатора
z_dim = 100
generator = Generator(z_dim).to(device)  # Переміщуємо генератор на GPU або CPU
discriminator = Discriminator().to(device)  # Переміщуємо дискримінатор на GPU або CPU

# Функція втрат та оптимізатори
criterion = nn.BCELoss()
lr = 0.0002
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Функція для збереження зображень
def save_generated_images(epoch, fixed_noise):
    with torch.no_grad():
        generated_images = generator(fixed_noise)
        generated_images = generated_images.detach().cpu().numpy()
        generated_images = (generated_images + 1) / 2  # Перетворення в діапазон [0, 1]

        fig, axes = plt.subplots(1, 8, figsize=(12, 12))
        for i, ax in enumerate(axes):
            ax.imshow(np.transpose(generated_images[i], (1, 2, 0)).squeeze(), cmap='gray')
            ax.axis('off')
        plt.savefig(f'generated_images_epoch_{epoch}.png')
        plt.close()

# Навчання GAN
num_epochs = 50
fixed_noise = torch.randn(8, z_dim, device=device)  # Фіксований шум на тому ж пристрої

for epoch in range(num_epochs):
    for real_images, _ in data_loader:
        batch_size = real_images.size(0)

        # Переміщення реальних зображень та міток на правильний пристрій
        real_images = real_images.to(device)
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        # --- Навчання дискримінатора ---
        optimizer_d.zero_grad()

        # Прогноз дискримінатора на реальних зображеннях
        output_real = discriminator(real_images)
        d_loss_real = criterion(output_real, real_labels)

        # Генерація фейкових зображень
        noise = torch.randn(batch_size, z_dim, device=device)
        fake_images = generator(noise)

        # Прогноз дискримінатора на згенерованих зображеннях
        output_fake = discriminator(fake_images.detach())
        d_loss_fake = criterion(output_fake, fake_labels)

        # Всього втрат для дискримінатора
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_d.step()

        # --- Навчання генератора ---
        optimizer_g.zero_grad()

        # Прогноз дискримінатора на згенерованих зображеннях (генератор намагається обдурити дискримінатор)
        output_fake_for_g = discriminator(fake_images)
        g_loss = criterion(output_fake_for_g, real_labels)  # Мітка 1 для генератора

        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    # Збереження зображень після кожної епохи
    save_generated_images(epoch + 1, fixed_noise)

print("Навчання завершено!")

Epoch [1/50], D Loss: 0.4901524782180786, G Loss: 1.808892011642456
Epoch [2/50], D Loss: 0.2605823278427124, G Loss: 3.7215170860290527
Epoch [3/50], D Loss: 0.1853657066822052, G Loss: 3.8130784034729004
Epoch [4/50], D Loss: 0.1945807933807373, G Loss: 3.834359645843506
Epoch [5/50], D Loss: 0.100370392203331, G Loss: 4.061067581176758
Epoch [6/50], D Loss: 0.5055583119392395, G Loss: 5.376664638519287
Epoch [7/50], D Loss: 0.5776491165161133, G Loss: 2.5038490295410156
Epoch [8/50], D Loss: 0.7711988091468811, G Loss: 1.6287810802459717
Epoch [9/50], D Loss: 0.9833308458328247, G Loss: 1.2963560819625854
Epoch [10/50], D Loss: 0.6708999872207642, G Loss: 1.6636345386505127
Epoch [11/50], D Loss: 0.7493727207183838, G Loss: 1.5428266525268555
Epoch [12/50], D Loss: 0.6954547166824341, G Loss: 1.685975193977356
Epoch [13/50], D Loss: 0.757044792175293, G Loss: 2.274993896484375
Epoch [14/50], D Loss: 0.7424869537353516, G Loss: 1.778397798538208
Epoch [15/50], D Loss: 0.9466207027435

In [8]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os

# Функція для збереження та візуалізації зображень
def save_generated_images(epoch, fixed_noise, generator, device, output_dir="generated_images"):
    with torch.no_grad():
        # Генерація зображень з фіксованим шумом
        generated_images = generator(fixed_noise).detach().cpu()
        generated_images = (generated_images + 1) / 2  # Перетворення в діапазон [0, 1]

        # Створення директорії для збереження, якщо її ще немає
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Збереження зображень
        fig, axes = plt.subplots(1, 8, figsize=(12, 12))
        for i, ax in enumerate(axes):
            ax.imshow(np.transpose(generated_images[i], (1, 2, 0)).squeeze(), cmap='gray')
            ax.axis('off')
        plt.savefig(f'{output_dir}/generated_images_epoch_{epoch}.png')
        plt.close()

# Збереження найкращої моделі генератора
def save_best_model(generator, epoch, g_loss, best_g_loss, model_dir="models"):
    if g_loss < best_g_loss:
        # Оновлення найкращої втрати та збереження моделі
        best_g_loss = g_loss
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        torch.save(generator.state_dict(), f'{model_dir}/best_generator.pth')
        print(f"Збережено найкращу модель генератора на епосі {epoch}")
    return best_g_loss

# Навчання GAN з візуалізацією та збереженням моделі
num_epochs = 50
fixed_noise = torch.randn(8, z_dim, device=device)  # Фіксований шум для візуалізації
best_g_loss = float('inf')  # Ініціалізація найкращої втрати для генератора

for epoch in range(num_epochs):
    for real_images, _ in data_loader:
        batch_size = real_images.size(0)

        # Переміщення даних на пристрій
        real_images = real_images.to(device)
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        # --- Навчання дискримінатора ---
        optimizer_d.zero_grad()

        # Прогноз на реальних зображеннях
        output_real = discriminator(real_images)
        d_loss_real = criterion(output_real, real_labels)

        # Генерація фейкових зображень
        noise = torch.randn(batch_size, z_dim, device=device)
        fake_images = generator(noise)

        # Прогноз на згенерованих зображеннях
        output_fake = discriminator(fake_images.detach())
        d_loss_fake = criterion(output_fake, fake_labels)

        # Загальні втрати для дискримінатора
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_d.step()

        # --- Навчання генератора ---
        optimizer_g.zero_grad()

        # Прогноз для генератора (ми хочемо, щоб дискримінатор приймав фейкові зображення за реальні)
        output_fake_for_g = discriminator(fake_images)
        g_loss = criterion(output_fake_for_g, real_labels)  # Мітка 1 для генератора

        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    # Збереження зображень після кожної епохи
    if (epoch + 1) % 5 == 0:  # Збереження зображень кожні 5 епох
        save_generated_images(epoch + 1, fixed_noise, generator, device)

    # Оновлення найкращої моделі генератора
    best_g_loss = save_best_model(generator, epoch + 1, g_loss.item(), best_g_loss)

print("Навчання завершено!")

Epoch [1/50], D Loss: 0.9642716646194458, G Loss: 1.5269559621810913
Збережено найкращу модель генератора на епосі 1
Epoch [2/50], D Loss: 0.8828217387199402, G Loss: 2.5503602027893066
Epoch [3/50], D Loss: 1.0358697175979614, G Loss: 1.6038002967834473
Epoch [4/50], D Loss: 1.0999438762664795, G Loss: 2.5469884872436523
Epoch [5/50], D Loss: 0.8785251379013062, G Loss: 2.037558078765869
Epoch [6/50], D Loss: 0.9338296055793762, G Loss: 2.1992149353027344
Epoch [7/50], D Loss: 1.0180569887161255, G Loss: 1.1594810485839844
Збережено найкращу модель генератора на епосі 7
Epoch [8/50], D Loss: 1.0071702003479004, G Loss: 1.853919506072998
Epoch [9/50], D Loss: 0.9061351418495178, G Loss: 1.9339609146118164
Epoch [10/50], D Loss: 0.9154525399208069, G Loss: 1.9707765579223633
Epoch [11/50], D Loss: 0.9796486496925354, G Loss: 1.4361650943756104
Epoch [12/50], D Loss: 1.4164085388183594, G Loss: 1.022302269935608
Збережено найкращу модель генератора на епосі 12
Epoch [13/50], D Loss: 0.98